In [3]:
%pip install arxiv

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [5]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

print(wiki.invoke({"query":"Langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain


In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama.embeddings import OllamaEmbeddings

ollama_embeddings=OllamaEmbeddings(model="nomic-embed-text")

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,ollama_embeddings)

retriever=vectordb.as_retriever
retriever



USER_AGENT environment variable not set, consider setting it to identify your requests.


<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x00000244C9952A10>>

In [7]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmth_search",
                                     "search for information about langsmith, for any questions about langsmith, you must use this tool")

In [8]:
retriever_tool.name

'langsmth_search'

In [9]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [10]:
tools=[wiki,arxiv,retriever_tool]


In [11]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'e:\\Langchain_Prac\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmth_search', description='search for information about langsmith, for any questions about langsmith, you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000244A920C860>, retriever=<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x0

In [12]:
from dotenv import load_dotenv
import os
from langchain_ollama.llms import OllamaLLM
load_dotenv()

llm=OllamaLLM(model="llama3.2")


In [40]:
# """


# from langchain import hub
# prompt=hub.pull("hwchase17/openai-functions-agent")
# prompt.messages

# from langchain_ollama import ChatOllama

# llm=ChatOllama(model="llama3.2")
# prompt = """
# Your task is to generate a response to a user query, leveraging the provided context.
# Here's the query: {input_variables}

# Here's the context: {context}

# **Guidelines:**

# * **Relevance:** Ensure your response directly addresses the query.
# * **Informativeness:** Provide comprehensive and informative answers.
# * **Clarity:** Use clear and concise language.
# * **Conciseness:** Avoid unnecessary verbosity.

# **Response:**
# """
# """


from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template("""
You are a helpful AI assistant. Your goal is to help the user by using the available tools.
To use a tool, output a JSON blob with "action" and "action_input" keys, like:
{{"action": "tool name", "action_input": "input to tool"}}

You will get back a response from the tool to help answer the user's question.

Here is the user's question: {input}

{agent_scratchpad}
""")



In [41]:
from langchain.agents import create_tool_calling_agent
#prompt.format(input="Provide detailed answer",context=retriever,agent_scratchpad="")
agent=create_tool_calling_agent(llm,tools,prompt)



In [42]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor



AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, template='\nYou are a helpful AI assistant. Your goal is to help the user by using the available tools.\nTo use a tool, output a JSON blob with "action" and "action_input" keys, like:\n{{"action": "tool name", "action_input": "input to tool"}}\n\nYou will get back a response from the tool to help answer the user\'s question.\n\nHere is the user\'s question: {input}\n\n{agent_scratchpad}\n'), additional_kwargs={})])
| RunnableBinding(bound=ChatOllama(model='llama3.2'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'wikipedia', 'description': 'A wr

In [43]:
agent_executor.invoke({"input":"tell me about audi"})




> Entering new AgentExecutor chain...


ResponseError: invalid format: expected "json" or a JSON schema